# Embedding & Vector Store

In [1]:
import config
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_chroma import Chroma
import pandas as pd
import os

c:\Users\Playdata\OneDrive\코딩\SK\13_Langchain\project\config.py:20: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model=embedding_model_name)


In [2]:
# setting

MODEL_NAME  = config.model_name
EMBEDDING_NAME = config.embedding_name

COLLECTION_NAME = config.collection_name
PERSIST_DIRECTORY = config.persist_directory

In [5]:
import pandas as pd
data = pd.read_csv('data/final_restaurant.csv', low_memory=False)
data.head()

,Unnamed: 0,id,예약,음식종류,식당이름,리본개수,전화번호,영업시간_평일,영업종료_평일,영업시간_주말,...,리뷰_상세,리뷰_요약,리뷰_평가,좌석_수,룸_수,체인점여부,리뉴얼여부,대표점여부,지점여부,웹사이트
0,0,4627,예약가능,"중식, 일반중식",홍보각,3,02-531-6479,미등록,미등록,미등록,...,국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스...,국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스...,정보 없음,64,"6개(2, 3, 4, 6, 8, 12석)",False,False,정보 없음,정보 없음,https://www.instagram.com/hongbogak_official/
1,1,29083,예약가능,"컨템포러리, 이노베이티브퀴진, 분자요리, 이탈리아식",쵸이닷,3,02-518-0318,12:00,22:30,미등록,...,최현석 셰프의 크리에이티브한 요리를 즐길 수 있는 레스토랑. 분자요리와 이탈리아 요...,최현석 셰프의 크리에이티브한 요리를 즐길 수 있는 레스토랑. 분자요리와 이탈리아 요...,“어디까지 발전할 수 있을지 진심으로 기대되는 곳.” “음식 너무나 재밌었어요.”,미등록,미등록,False,False,정보 없음,정보 없음,https://www.instagram.com/choidot_official/
2,2,28946,예약가능,프랑스식,더그린테이블,3,02-591-2672,미등록,미등록,미등록,...,김은희 셰프가 운영하는 프렌치 레스토랑. 사계절에 따라 변화하는 우리나라 식재료로 ...,김은희 셰프가 운영하는 프렌치 레스토랑. 사계절에 따라 변화하는 우리나라 식재료로 ...,“음식의 결 하나하나에서 섬세한 정성이 듬뿍 느껴진다. 신선한 재료들이 훌륭하게 조...,미등록,미등록,False,False,정보 없음,정보 없음,"http://thegreentable.co.kr/, https://www.insta..."
3,3,4963,예약가능,"한식(육류), 소갈비, 소고기구이, 한우오마카세",명월관,3,02-6330-9050,미등록,미등록,미등록,...,"워커힐호텔 내 한옥 분위기의 별채로 있는, 주변 경치가 아름다운 숯불구이 전문점. ...","워커힐호텔 내 한옥 분위기의 별채로 있는, 주변 경치가 아름다운 숯불구이 전문점. ...",정보 없음,300,4개,False,False,정보 없음,정보 없음,https://www.walkerhill.com/grandwalkerhillseou...
4,4,859,예약가능,프랑스식,콘티넨탈,3,02-2230-3369,미등록,미등록,미등록,...,"정통 유럽 스타일의 프렌치 레스토랑으로, 40여 년의 역사를 자랑하는, 서울에서 오...","정통 유럽 스타일의 프렌치 레스토랑으로, 40여 년의 역사를 자랑하는, 서울에서 오...",“최고의 레스토랑이다.” “분위기가 상당히 모던하고 세련된 느낌. 재료의 질이 최상...,70석,1개,False,False,정보 없음,정보 없음,https://www.shilla.net/seoul/dining/viewDining...


In [ ]:
####################################
# 데이터 불러오기 및 저장
####################################
data = pd.read_csv('data/final_restaurant.csv', low_memory=False)


# 모든 데이터를 활용하도록 문서화

from langchain_core.documents import Document
# raw_df = data.head()
doc_list = []
for _, info in data.iterrows():
    doc_list.append(Document(page_content=str(dict(info)), metadata=dict(info)))



print(f"총 {len(doc_list)}개의 문서가 생성되었습니다.")


# 분리
# splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
#     model_name=MODEL_NAME, 
#     chunk_size=CHUNK_SIZE,
#     chunk_overlap=CHUNK_OVERLAP
# )
# splited_docs = splitter.split_documents(documents)

# print("데이터 분리 완료:", len(splited_docs), end='\n\n')


# Vector store 저장
embedding_model = OpenAIEmbeddings(
    model=EMBEDDING_NAME
)

# Persist directory 없는 경우 생성
if not os.path.exists(PERSIST_DIRECTORY):
    os.makedirs(PERSIST_DIRECTORY)

# 연결 + document 추가
vector_store = Chroma.from_documents(
    documents= doc_list,
    embedding=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

print("vecor_store에 저장완료")

총 17280개의 문서가 생성되었습니다.
vecor_store에 splited_docs 저장완료


In [8]:
doc_list[0].page_content

"{'Unnamed: 0': 0, 'id': 4627, '예약': '예약가능', '음식종류': '중식, 일반중식', '식당이름': '홍보각', '리본개수': 3, '전화번호': '02-531-6479', '영업시간_평일': '미등록', '영업종료_평일': '미등록', '영업시간_주말': '미등록', '영업종료_주말': '미등록', '휴무일': '연중무휴', '앱사용여부': True, '주차가능여부': '가능', '카드결제가능여부': 'y', '방문가능여부': nan, '메뉴정보': '스페셜코스(1인 15만원), 디너코스(A 12만원, B 16만원), 고법불도장(13만원), 모자새우(소 6만원, 대 10만원), 한알탕수육(소 4만8천원, 대 7만원), 특선냉채(2만8천원), 셰프특선수프(3만8천원), 셰프스페셜송이특찜(15만원)', '가격대': '10~15만원대', '영업시간': '12:00~15:00/18:00~21:30(마지막 주문 20:00) ', '신규오픈날짜': '2007년 ', '도로명주소': '서울특별시 강남구 봉은사로 130', '시': '서울특별시', '구': '강남구', '도로명': '동호로', '리뷰_상세': '국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스토랑. 고급스러운 분위기의 인테리어에서 정통 중식을 맛볼 수 있다. 여경래 셰프의 시그니처 메뉴인 불도장, 모자새우 등을 선보인다.', '리뷰_요약': '국내 중식의 대가인 여경래 셰프가 2023년 이전하여 오픈한 중식 파인 다이닝 레스토랑. 고급스러운 분위기의 인테리어에서 정통 중식을 맛볼 수 있다. 여경래 셰프의 시그니처 메뉴인 불도장, 모자새우 등을 선보인다.', '리뷰_평가': '정보 없음', '좌석_수': '64', '룸_수': '6개(2, 3, 4, 6, 8, 12석)', '체인점여부': False, '리뉴얼여부': False, '대표점여부': '정보 없음', '지점여부': '정보 없음', '웹사이트': 'https://www.instagram